In [12]:
import os
from pyspark import SparkContext, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.types import (
    StructType,
    StructField,
    IntegerType,
    StringType,
    FloatType,
    Row
)
from pyspark.sql.functions import col

# Set parent directory

In [2]:
base = os.path.abspath(os.pardir)
data_dir = os.path.join(base, "work", "files")

# Spark Context

In [3]:
spark = SparkContext(master="local", appName="SparkProject")
sql_context = SQLContext(spark)

21/08/21 01:45:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


# Schemas

In [4]:
schemas = {
    "deporte": StructType([
        StructField("deporte_id", IntegerType(), False),
        StructField("deporte", StringType(), False)
    ]),
    "deportista": StructType([
        StructField("deportista_id", IntegerType(), False),
        StructField("nombre", StringType(), False),
        StructField("genero", StringType(), False),
        StructField("edad", IntegerType(), False),
        StructField("altura", IntegerType(), False),
        StructField("peso", FloatType(), False),
        StructField("equipo_id", IntegerType(), False)
    ]),
    "evento": StructType([
        StructField("evento_id", IntegerType(), False),
        StructField("evento", StringType(), False),
        StructField("deporte_id", IntegerType(), False)
    ]),
    "juegos": StructType([
        StructField("juego_id", IntegerType(), False),
        StructField("nombre_juego", StringType(), False),
        StructField("anio", IntegerType(), False),
        StructField("temporada", StringType(), False),
        StructField("ciudad", StringType(), False)
    ]),
    "paises": StructType([
        StructField("pais_id", IntegerType(), False),
        StructField("equipo", StringType(), False),
        StructField("sigla", StringType(), False)
    ]),
    "resultados": StructType([
        StructField("resultado_id", IntegerType(), False),
        StructField("medalla", StringType(), False),
        StructField("deportista_id", IntegerType(), False),
        StructField("juego_id", IntegerType(), False),
        StructField("evento_id", IntegerType(), False)
    ])
}

# Load Dataframes

In [5]:
dp1 = sql_context.read.schema(schemas['deportista']).option("header", "true").csv(f"{data_dir}/deportista.csv")
dp2 = sql_context.read.schema(schemas['deportista']).option("header", "false").csv(f"{data_dir}/deportista2.csv")

dataframes = {
    "deporte": sql_context.read.schema(schemas['deporte']).option("header", "true").csv(f"{data_dir}/deporte.csv"),
    "deportista": dp1.union(dp2),
    "evento": sql_context.read.schema(schemas['evento']).option("header", "true").csv(f"{data_dir}/evento.csv"),
    "juegos": sql_context.read.schema(schemas['juegos']).option("header", "true").csv(f"{data_dir}/juegos.csv"),
    "paises": sql_context.read.schema(schemas['paises']).option("header", "true").csv(f"{data_dir}/paises.csv"),
    "resultados": sql_context.read.schema(schemas['resultados']).option("header", "true").csv(f"{data_dir}/resultados.csv")
}
del(dp1, dp2)

In [7]:
dataframes['deportista'].printSchema()

root
 |-- deportista_id: integer (nullable = true)
 |-- nombre: string (nullable = true)
 |-- genero: string (nullable = true)
 |-- edad: integer (nullable = true)
 |-- altura: integer (nullable = true)
 |-- peso: float (nullable = true)
 |-- equipo_id: integer (nullable = true)



# Transform

In [8]:
dataframes['deportista'] = dataframes['deportista'].withColumnRenamed("genero", "sexo")

In [9]:
dataframes['deportista'] = dataframes['deportista'].drop("altura")

In [11]:
dataframes['deportista'].printSchema()

root
 |-- deportista_id: integer (nullable = true)
 |-- nombre: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- edad: integer (nullable = true)
 |-- peso: float (nullable = true)
 |-- equipo_id: integer (nullable = true)



- La forma anterior de renombrar lo hace recorriendo todo la columna, lo cual puede ser costoso. Para un mejor rendimiento, es mejor usar la siguiente forma.

In [13]:
dataframes['deportista'] = dataframes['deportista'].select(
    "deportista_id",
    "nombre",
    col("edad").alias("edad_al_jugar"),
    "equipo_id"
)

In [15]:
dataframes['deportista'].show(5)

+-------------+--------------------+-------------+---------+
|deportista_id|              nombre|edad_al_jugar|equipo_id|
+-------------+--------------------+-------------+---------+
|            1|           A Dijiang|           24|      199|
|            2|            A Lamusi|           23|      199|
|            3| Gunnar Nielsen Aaby|           24|      273|
|            4|Edgar Lindenau Aabye|           34|      278|
|            5|Christine Jacoba ...|           21|      705|
+-------------+--------------------+-------------+---------+
only showing top 5 rows



In [16]:
dataframes['deportista'].sort("edad_al_jugar").show()

+-------------+--------------------+-------------+---------+
|deportista_id|              nombre|edad_al_jugar|equipo_id|
+-------------+--------------------+-------------+---------+
|          224|     Mohamed AbdelEl|            0|      308|
|          487|      Inni Aboubacar|            0|      721|
|          226|Sanad Bushara Abd...|            0|     1003|
|           58|    Georgi Abadzhiev|            0|      154|
|          230|    Moustafa Abdelal|            0|      308|
|          102|   Sayed Fahmy Abaza|            0|      308|
|          260|  Ahmed Abdo Mustafa|            0|     1003|
|          139|George Ioannis Abbot|            0|     1043|
|          281|      S. Abdul Hamid|            0|      487|
|          163|     Ismail Abdallah|            0|     1095|
|          285|Talal Hassoun Abd...|            0|      497|
|          173| Mohamed Abdel Fatah|            0|     1003|
|          179|Ibrahim Saad Abde...|            0|     1003|
|          378|     Ange

- Como podemos ver, tenemos información basura ya que no es posible tener 0 años

In [20]:
dataframes['deportista'] = dataframes['deportista'].filter((dataframes['deportista'].edad_al_jugar != 0))

In [21]:
dataframes['deportista'].sort("edad_al_jugar").show()

+-------------+--------------------+-------------+---------+
|deportista_id|              nombre|edad_al_jugar|equipo_id|
+-------------+--------------------+-------------+---------+
|        71691|  Dimitrios Loundras|           10|      333|
|        70616|          Liu Luyang|           11|      199|
|       118925|Megan Olwen Deven...|           11|      413|
|        52070|        Etsuko Inada|           11|      514|
|        22411|Magdalena Cecilia...|           11|      413|
|        40129|    Luigina Giavotti|           11|      507|
|        47618|Sonja Henie Toppi...|           11|      742|
|        76675|   Marcelle Matthews|           11|      967|
|        37333|Carlos Bienvenido...|           11|      982|
|        51268|      Beatrice Hutiu|           11|      861|
|       126307|        Liana Vicens|           11|      825|
|        48939|             Ho Gang|           12|      738|
|        49142|        Jan Hoffmann|           12|      302|
|        42835|   Werner

# Joins

In [29]:
report = dataframes['deportista'].join(
    dataframes['resultados'],
    dataframes['deportista'].deportista_id == dataframes['resultados'].deportista_id,
    "left"
).join(
    dataframes['juegos'],
    dataframes['resultados'].juego_id == dataframes['juegos'].juego_id,
    "left"
).join(
    dataframes['evento'],
    dataframes['resultados'].evento_id == dataframes['evento'].evento_id,
    "left"
).join(
    dataframes['deporte'],
    dataframes['evento'].deporte_id == dataframes['deporte'].deporte_id,
    "left"
).select(
    dataframes['deportista'].nombre, "edad_al_jugar", "medalla", col("anio").alias("año de juego"),
    dataframes['deporte'].deporte.alias("deporte")
)

In [30]:
report.show()

21/08/21 02:26:02 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , annio
 Schema: juego_id, anio
Expected: juego_id but found: 
CSV file: file:///home/jovyan/work/files/juegos.csv


+--------------------+-------------+-------+------------+--------------------+
|              nombre|edad_al_jugar|medalla|año de juego|             deporte|
+--------------------+-------------+-------+------------+--------------------+
|           A Dijiang|           24|     NA|        1992|          Basketball|
|            A Lamusi|           23|     NA|        2012|                Judo|
| Gunnar Nielsen Aaby|           24|     NA|        1920|            Football|
|Edgar Lindenau Aabye|           34|   Gold|        1900|          Tug-Of-War|
|Christine Jacoba ...|           21|     NA|        1994|       Speed Skating|
|Christine Jacoba ...|           21|     NA|        1994|       Speed Skating|
|Christine Jacoba ...|           21|     NA|        1992|       Speed Skating|
|Christine Jacoba ...|           21|     NA|        1992|       Speed Skating|
|Christine Jacoba ...|           21|     NA|        1988|       Speed Skating|
|Christine Jacoba ...|           21|     NA|        

In [33]:
report.filter((report.medalla != "NA")).show()

21/08/21 02:27:07 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , annio
 Schema: juego_id, anio
Expected: juego_id but found: 
CSV file: file:///home/jovyan/work/files/juegos.csv


+--------------------+-------------+-------+------------+-------------+
|              nombre|edad_al_jugar|medalla|año de juego|      deporte|
+--------------------+-------------+-------+------------+-------------+
|Edgar Lindenau Aabye|           34|   Gold|        1900|   Tug-Of-War|
|Arvo Ossian Aaltonen|           22| Bronze|        1920|     Swimming|
|Arvo Ossian Aaltonen|           22| Bronze|        1920|     Swimming|
|Juhamatti Tapio A...|           28| Bronze|        2014|   Ice Hockey|
|Paavo Johannes Aa...|           28| Bronze|        1948|   Gymnastics|
|Paavo Johannes Aa...|           28|   Gold|        1948|   Gymnastics|
|Paavo Johannes Aa...|           28|   Gold|        1948|   Gymnastics|
|Paavo Johannes Aa...|           28|   Gold|        1948|   Gymnastics|
|Paavo Johannes Aa...|           28| Bronze|        1952|   Gymnastics|
|  Kjetil Andr Aamodt|           20|   Gold|        1992|Alpine Skiing|
|  Kjetil Andr Aamodt|           20| Bronze|        1992|Alpine 

In [ ]:
paises_medallas = dataframes['deportista'].join(
    dataframes['resultados'],
    dataframes['deportista'].deportista_id == dataframes['resultados'].deportista_id,
    "left"
).join(
    dataframes['juegos'],
    dataframes['resultados'].juego_id == dataframes['juegos'].juego_id,
    "left"
).join(
    dataframes['evento'],
    dataframes['resultados'].evento_id == dataframes['evento'].evento_id,
    "left"
).join(
    dataframes['deporte'],
    dataframes['evento'].deporte_id == dataframes['deporte'].deporte_id,
    "left"
).select(
    dataframes['deportista'].nombre, "edad_al_jugar", "medalla", col("anio").alias("año de juego"),
    dataframes['deporte'].deporte.alias("deporte")
)